# Python Function Generator

* This notebook demonstrates prompt-engineering techniques and practical prompt-engineering code snippets needed for building agent-like behavior:
    * **Prompt chaining** - Breaking complex tasks into sequential steps
    * **Context management** - Controlling what the LLM "remembers"
    * **Role assignment** - Using system messages to set behavior
    * **Output formatting** - Requesting specific formats (code blocks)
    * **Memory manipulation** - Editing conversation history to shape future responses

In [1]:
# FIRST TIME SETUP (run these commands in terminal, not in notebook):
# 1. Create virtual environment: python3 -m venv venv
# 2. Activate it: source venv/bin/activate  (macOS/Linux) or venv\Scripts\activate (Windows)
# 3. Install dependencies: pip install -r requirements.txt
# 4. Create .env file with: OPENAI_API_KEY=sk-your_key_here
# 5. Start Jupyter: jupyter notebook

from dotenv import load_dotenv
import os

# Load environment variables from .env file
env_loaded = load_dotenv()

if env_loaded:
    print("✓ .env file loaded successfully")
else:
    print("✗ .env file not found")

# Get the API key from environment
api_key = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key

# Validate key exists and has correct format
if api_key and api_key.startswith('sk-'):
    print("✓ API key loaded successfully")
else:
    print("✗ API key not found or invalid format")

✓ .env file loaded successfully
✓ API key loaded successfully


In [2]:
from litellm import completion
from typing import List, Dict

In [3]:
# Write a function to handle LLM interactions
def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get response"""
    response = completion(
        model="openai/gpt-4o",
        messages=messages,
        max_tokens=1024
    )
    return response.choices[0].message.content

# Define a function that extracts code returned by response and returns it as a string
def extract_code_block(response: str) -> str:
    """Extract clean code from LLM response"""
    # If response does not contain code block markers, return response as is
    if not '```' in response:
        return response
    
    # Extract code between first pair of triple backticks
    code_block = response.split('```')[1].strip()
    
    # LLMs often return markdown with 'python' at the start of the returned code. If present, remove it
    if code_block.startswith("python"):
        code_block = code_block[6:]
    
    return code_block

def develop_custom_function():
    # Get all executed notebook code as context
    from IPython import get_ipython
    ipython = get_ipython()
    notebook_context = '\n\n'.join([code for _, _, code in ipython.history_manager.get_range(output=False)])
    
    # Ask the user what kind of function they would like to create
    print("\nEnter a description of what kind of function would you like to create?")
    print("Enter your description: ", end='')
    function_description = input().strip()
    
    # Initialize conversation with system prompt - this sets the LLM's role/personality
    messages = [
        {"role": "system", "content": "You are a Python expert. Be concise and professional. Output ONLY code with minimal comments."}
    ]
    
    # ===== FIRST PROMPT: Write the basic function =====
    messages.append({
        "role": "user",
        "content": f"""Here is the code executed so far in this notebook:
```python
{notebook_context}
```

Write a Python function that {function_description}. Make it compatible with the existing code above. This function will be called in the next cell and must run successfully without errors. 

IMPORTANT: If any existing code needs to be modified for this function to work (e.g., function signatures, return values, global variables), output those modifications FIRST in a separate code block labeled "REQUIRED MODIFICATIONS:", followed by the new function definition.

Output in ```python code block``` format. No examples, no usage code."""
    })
    
    initial_function = generate_response(messages)
    initial_function = extract_code_block(initial_function)
    messages.append({"role": "assistant", "content": "```python\n\n"+initial_function+"\n\n```"})
    
    # ===== SECOND PROMPT: Add documentation =====
    messages.append({
        "role": "user",
        "content": "Add concise documentation to this function (6 sentences maximum). Include: brief description, parameters, returns, and one example. Output in a ```python code block```."
    })
    
    documentation = generate_response(messages)
    documentation = extract_code_block(documentation)
    messages.append({"role": "assistant", "content": "```python\n\n"+documentation+"\n\n```"})
    
    # ===== THIRD PROMPT: Create test cases =====
    messages.append({
        "role": "user",
        "content": "Add unittest test cases. Include ONE single-line comment per test explaining what it tests. No multi-line comments, no docstrings in tests, no example usage. Output in a ```python code block```."
    })
    
    test_cases = generate_response(messages)
    test_cases = extract_code_block(test_cases)
    
    # Print output
    print("\n" + "="*50)
    print("Function:")
    print("="*50)
    print(initial_function)
    
    if "REQUIRED MODIFICATIONS" in initial_function or "modify" in initial_function.lower():
        print("\n" + "="*50)
        print("NOTE: Check if existing code needs updates")
        print("="*50)
    
    print("\n" + "="*50)
    print("Documentation:")
    print("="*50)
    print(documentation)
    
    print("\n" + "="*50)
    print("Test Cases:")
    print("="*50)
    print(test_cases)
    
    return documentation, test_cases, messages

if __name__ == "__main__":
    function_code, tests, messages = develop_custom_function()


Enter a description of what kind of function would you like to create?
Enter your description: 

 A function that prints a sum of the total number of tokens consumed by the most recent api call (should print a real number and not an estimate)



Function:

REQUIRED MODIFICATIONS:
from litellm import completion, get_usage

NEW FUNCTION:
def print_last_api_call_tokens():
    """Prints the total number of tokens consumed by the most recent API call."""
    usage = get_usage()
    total_tokens = usage['total_tokens']
    print(total_tokens)

NOTE: Check if existing code needs updates

Documentation:

def print_last_api_call_tokens():
    """
    Prints the total number of tokens consumed by the most recent API call.

    Parameters: 
        None

    Returns:
        None

    Example:
        print_last_api_call_tokens()  # Outputs the total token count from the last API call
    """

Test Cases:



import unittest
from unittest.mock import patch

class TestPrintLastApiCallTokens(unittest.TestCase):

    @patch('litellm.get_usage', return_value={'total_tokens': 150})
    def test_correct_token_output(self, mock_get_usage):
        # Tests if function prints correct token count
        with patch('builtins.print') as mock_print: